In [1]:
import sys
import numpy as np
sys.path.append('../')
from deepod.models.time_series import TranAD as model
from load_data import load_data
from deepod.metrics import ts_metrics
from deepod.metrics import point_adjustment 

In [2]:
train_imu,train_audio,train_spec,val_imu,val_audio,val_spec,val_imu_normal,val_audio_normal,val_spec_normal = load_data()

train_combine = np.concatenate([np.mean(train_audio,axis=-1),train_imu],axis=1)

test_audio_all = np.concatenate([val_audio_normal,val_audio],axis=0)
test_imu_all   = np.concatenate([val_imu_normal,val_imu],axis=0)
test_combine = np.concatenate([np.mean(test_audio_all,axis=-1),test_imu_all],axis=1)
labels_all = np.concatenate([np.zeros(val_audio_normal.shape[0]),np.ones(val_audio.shape[0])])

In [3]:
train_audio = np.mean(train_audio,axis=-1)
train_audio = train_audio.reshape(-1,1)

In [4]:
# train_imu = train_imu[:,:,np.newaxis]

In [5]:
# train_imu = train_imu.reshape(-1,1)
# test_imu_all = test_imu_all.reshape(-1,1)

In [4]:
clf_imu = model(epochs=50,batch_size=4,seq_len=1,stride=1)
clf_imu.fit(train_imu)
clf_audio = model(epochs=50,batch_size=2,seq_len=1,stride=1)
clf_audio.fit(train_audio)
clf_all = model(epochs=50,batch_size=4,seq_len=1,stride=1)
clf_all.fit(train_combine)

/home/iot/anaconda3/envs/ai/lib/python3.8/site-packages/torch/nn/modules/transformer.py:306: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer was not TransformerEncoderLayer
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Epoch 1,	 L1 = 0.050824097929183736
Epoch 2,	 L1 = 0.0232653664001687
Epoch 3,	 L1 = 0.018198362932625143
Epoch 4,	 L1 = 0.021501501090824604
Epoch 5,	 L1 = 0.018315152000551196
Epoch 6,	 L1 = 0.02985290742733262
Epoch 7,	 L1 = 0.032990856942805374
Epoch 8,	 L1 = 0.033986042592335834
Epoch 9,	 L1 = 0.04739513586867939
Epoch 10,	 L1 = 0.05270653150298379
Epoch 11,	 L1 = 0.05392142791639675
Epoch 12,	 L1 = 0.051397584209387955
Epoch 13,	 L1 = 0.031845593062991444
Epoch 14,	 L1 = 0.03035657099363479
Epoch 15,	 L1 = 0.032225117849355396
Epoch 16,	 L1 = 0.03327802213078195
Epoch 17,	 L1 = 0.034288524904034355
Epoch 18,	 L1 = 0.029933573678135872
Epoch 19,	 L1 = 0.028407635844566605
Epoch 20,	 L1 = 0.029421935111961582
Epoch 21,	 L1 = 0.028199359943920917
Epoch 22,	 L1 = 0.02920049191875891
Epoch 23,	 L1 = 0.02844747706231746
Epoch 24,	 L1 = 0.02950038498436863
Epoch 25,	 L1 = 0.027387919327752155
Epoch 26,	 L1 = 0.027531089176508514
Epoch 27,	 L1 = 0.029775227504697712
Epoch 28,	 L1 = 0.025

: 

In [ ]:
scores_audio      = clf_audio.decision_function(np.mean(test_audio_all,axis=-1).reshape(-1,1))
scores_imu        = clf_imu.decision_function(test_imu_all.reshape(-1,1))
scores_all        = clf_all.decision_function(test_combine)

In [ ]:
#auc, mean precision, F1, percision, recall
eval_metrics_audio = ts_metrics(labels_all, scores_audio)
eval_metrics_imu = ts_metrics(labels_all, scores_imu)
result_audio = eval_metrics_audio
result_imu = eval_metrics_imu
eval_metrics_all = ts_metrics(labels_all, scores_all)
result_all = eval_metrics_all

print(result_audio)
print(result_imu)
print(result_all)

(0.5159536236013262, 0.460431664787213, 0.556200364363793, 0.3852376137512639, 1.0)
